## Old Functions and Imports

In [ ]:
%load_ext lab_black

import h5py
import os

from dataclasses import dataclass
from tqdm.auto import tqdm
from scipy.signal import savgol_filter
from scipy.interpolate import interp2d
from functools import lru_cache
import lmfit as lm

from typing import Dict, List, Optional, Tuple
import numpy as np
import plotly.graph_objects as go
import plotly.colors as pc
import matplotlib.pyplot as plt


import sys

sys.path.append(r"C:\Users\atully\Code\GitHub\ARPES Code\arpes-code-python")
from arpes_functions import (
    fitting_functions,
    analysis_functions,
    plotting_functions,
    HDF5_loader,
    misc_functions,
    filter_functions,
    tr_functions,
    loading_functions,
    cnn,
)

colors = pc.qualitative.D3
colors_seq = pc.sequential.dense
angstrom = "\u212B"
Theta = "\u0398"

# Load Data

In [ ]:
ddir = r"E:\atully\arpes_data\2023_June\C60\ARPES\TR"
files = []

# files = [f"TR0/TR0_Ali_avg.h5"]
# files = [f"TR1/TR1_Ali_avg.h5"]
# files = [f"TR2/TR2_Ali_avg.h5"]
# files = [f"TR0&TR2_Ali_avg.h5"]

## High Statistics Scans -- at M to K##
# TR6: -0.5 to 0.5 ps
# TR7: -1 to 1 ps in steps of 100 fs, then out to 30 ps with variable steps

## High Statistics Scans -- at G to K ##
# TR9&TR11: -0.5 to 0.5 ps
# TR12: -1 to 1 ps in steps of 100 fs, then out to 30 ps with variable steps


files = [
    f"TR4_Ali_avg.h5",
    "TR6_Ali_avg.h5",
    "TR7_Ali_avg.h5",
    "TR9&TR11_Ali_avg.h5",
    "TR12_Ali_avg_best.h5",
]

### February Comparison ###
# ddir = r"E:\atully\arpes_data\2023_February\6eV\TR"
# files = []
# # # files = ["TR3_Ali_avg.h5"]
# # # files = [
# # #     "TR4_Ali_avg.h5"
# # # ]  # 2.6 eV center energy; -1 to 1 ps, same number of steps as first 2 ps of TR3
# # files = ["TR_001_1.h5"]

ARPES_DATA: Dict[str, tr_functions.ArpesData] = {}
ARPES_ATTRS: Dict[str, tr_functions.ArpesAttrs] = {}
for file in tqdm(files):
    data, kx, ky, energy = loading_functions.load_hdf5(ddir, file)
    ARPES_DATA[file] = tr_functions.ArpesData(
        data=data, theta=kx, phi_or_time=ky, energy=energy
    )
    ARPES_ATTRS[file] = tr_functions.load_attrs_hdf5(ddir, file)

In [ ]:
# ad = ARPES_DATA[files[0]]
# for k in ["energy", "theta", "phi_or_time"]:
#     print(f"{k}.shape = {getattr(ad, k).shape}")
# print(f"Data.shape = {ad.data.shape}")

In [ ]:
ad_4 = ARPES_DATA[files[0]]
ad_6 = ARPES_DATA[files[1]]
ad_7 = ARPES_DATA[files[2]]
ad_9_11 = ARPES_DATA[files[3]]
ad_12 = ARPES_DATA[files[4]]

# ad_feb_1 = ARPES_DATA[files[0]]
# ad_feb_4 = ARPES_DATA[files[0]]

In [ ]:
## Zero Delay, HOMO, EF ##

time_zero = 34.8225  # Bi2Se3

time_zero_feb = 37.96  # February

EF_400 = 1.91  # in kinetic energy, slit 400
EF_700 = 1.94  # in kinetic energy, slit 700

homo = -2.05

homo_400 = homo + EF_400
homo_700 = homo + EF_700

In [ ]:
ad_6.energy = ad_6.energy - homo_400  # M - K
ad_9_11.energy = ad_9_11.energy - homo_400  # G - K

In [ ]:
ad_6.phi_or_time = tr_functions.mm_to_ps(ad_6.phi_or_time, time_zero)
ad_9_11.phi_or_time = tr_functions.mm_to_ps(ad_9_11.phi_or_time, time_zero)

# Analysis Setup

In [ ]:
## Integrate over desired angular range ##

slice_dim = "x"
slice_val = 0
int_range = 20  # if this value is more that the integration range, my get_2D_slice function will just integrate over the max range.

xlim = None
ylim = None
x_bin = 1
y_bin = 1

In [ ]:
# ## Integrate over desired time range ##

# slice_dim = "z"
# slice_val = time_zero + tr_functions.ps_to_mm(0, time_zero)
# int_range = tr_functions.ps_to_mm(
#     0.5, time_zero
# )  # if this value is more that the integration range, my get_2D_slice function will just integrate over the max range.

# # xlim = (-20, 16.5)
# xlim = (-20, 15)
# ylim = None

# x_bin = 1
# y_bin = 1

# Analysis

In [ ]:
title = f"C<sub>60</sub> Pump Effect"
# title = f"CT<sub>2</sub>"
title = f"C<sub>60</sub> Pump Effect"

yaxis_title = "E - E<sub>HOMO</sub> (eV)"
xaxis_title = "Delay (ps)"
# xaxis_title=f"k<sub>x</sub> [{angstrom}<sup>-1</sup>]"

In [ ]:
all_vals = []
for ad in [ad_6, ad_9_11]:
    all_vals.append(
        tr_functions.slice_datacube(
            ad_dataclass=ad,
            slice_dim=slice_dim,
            slice_val=slice_val,
            int_range=int_range,
            xlim=xlim,
            ylim=(
                ad.energy[57],
                ad.energy[1007],
            ),  # get rid of zero padding on datasets
            x_bin=x_bin,
            y_bin=y_bin,
            norm_data=False,
            plot_data=False,
        )
    )

x6, y6, d6 = all_vals[0]
x911, y911, d911 = all_vals[1]

In [ ]:
## Plot Data ##
# x_plot, y_plot, d_plot = x6, y6, d6
x_plot, y_plot, d_plot = x911, y911, d911

# # FFT angular dispersion data
# d_2d = filter_functions.fft2d_mask(d_2d, plot=False)

fig = tr_functions.thesis_fig(
    title=title,
    xaxis_title=xaxis_title,
    yaxis_title=yaxis_title,
    equiv_axes=False,
    height=600,
    width=800,
    dtick_y=0.1,
)

fig.add_trace(
    go.Heatmap(
        x=x_plot,
        y=y_plot,
        z=analysis_functions.norm_data(d_plot),
        coloraxis="coloraxis",
    )
)

# fig.update_coloraxes(colorscale="greys", showscale=False)
# fig.update_coloraxes(cmin=0.0, cmax=0.8)

fig.show()

In [ ]:
## Integrated over energy ##

xlim = None
ylim = None
# ylim = (3.3, 3.545)

## Get slice
x_1d, row = tr_functions.get_1d_y_slice(
    x=x_plot,
    y=y_plot,
    data=d_plot,
    xlims=xlim,
    y_range=ylim,
)

## Plot data
if ylim is None:
    ylim = "all"
fig = tr_functions.thesis_fig(
    title=f"Average over {ylim} eV",
    xaxis_title="Delay (ps)",
    yaxis_title="Intensity (arb. u)",
    equiv_axes=False,
    gridlines=False,
    height=600,
    width=900,
)

fig.add_trace(
    go.Scatter(
        x=x_1d,
        y=row,
        name="data",
        mode="markers",
        marker=dict(symbol="cross", size=8),
    )
)

## For After Fit
# fig.add_trace(go.Scatter(x=x, y=fit.eval(x=x_1d), name="fit"))

x_ = np.linspace(min(x_1d), max(x_1d), 1000)
fig.add_trace(go.Scatter(x=x_, y=fit.eval(x=x_), name="fit"))

components = fit.eval_components(x=x_1d)
for model_name, model_value in list(components.items())[0:1]:
    fig.add_annotation(
        # x=fit.params[f"center"].value,
        # y=fit.eval(x=fit.params[f"{model_name}center"].value),
        xref="x domain",
        yref="y domain",
        x=0.01,
        y=0.99,
        showarrow=False,
        # text=f'Peak center: {fit.params[f"center"].value:.2f} +/- {fit.params[f"center"].stderr:.4f} mm<br>FWHM: {tr_functions.mm_to_ps(tr_functions.sig_to_fwhm(fit.params["sigma"].value)):.3f} +/- {tr_functions.mm_to_ps(tr_functions.sig_to_fwhm(fit.params[f"sigma"].stderr)):.4f} ps',
        text=f'Peak center:<br>{fit.params[f"center"].value:.2f} +/- {fit.params[f"center"].stderr:.4f} ps<br><br>FWHM:<br>{tr_functions.sig_to_fwhm(fit.params["sigma"].value):.3f} +/- {tr_functions.sig_to_fwhm(fit.params[f"sigma"].stderr):.4f} ps',
        # text=f'Peak:<br>{np.round(x_1d[np.where(fit.eval(x=x_1d) == np.max(fit.eval(x=x_1d)))][0], 2)} ps<br><br>FWHM:<br>{tr_functions.sig_to_fwhm(fit.params["sigma"].value):.3f} +/- {tr_functions.sig_to_fwhm(fit.params[f"sigma"].stderr):.4f} ps',
        font=dict(size=18),
    )

fig.show()

print(
    f'center: {np.round(fit.params["center"].value, 2)} +/- {fit.params[f"center"].stderr:.4f} ps'
)
print(
    f'decay: {np.round(1 / fit.params["gamma"].value, 3) * 1000} +/- {1 / fit.params[f"gamma"].stderr:.4f} fs'
)
# print(f"peak: {x_1d[np.where(fit.eval(x=x_1d) == np.max(fit.eval(x=x_1d)))][0]}")
print(
    f"peak: {np.round(x_1d[np.where(fit.eval(x=x_1d) == np.max(fit.eval(x=x_1d)))][0], 2)} ps"
)
print(
    f'FWHM: {tr_functions.sig_to_fwhm(fit.params["sigma"].value):.3f} +/- {tr_functions.sig_to_fwhm(fit.params[f"sigma"].stderr):.4f} ps'
)

In [ ]:
## Plot Residuals
fig = tr_functions.thesis_fig(
    title=f"Residuals",
    xaxis_title=xaxis_title,
    yaxis_title="Residuals",
    equiv_axes=False,
    gridlines=False,
    height=400,
    width=900,
)

fig.add_trace(go.Scatter(x=x_1d, y=row - fit.eval(x=x_1d), name="fit"))
fig.add_hline(y=0, line=dict(dash="dash"))

In [ ]:
from statsmodels.stats.stattools import durbin_watson

resids = row - fit.eval(x=x_1d)

dw = durbin_watson(resids)
print(f"Durbin-Watson: {dw}")

In [ ]:
## Fit ##

import lmfit as lm

offset_type = "constant"
x = x_1d
data = row

c = np.mean(data)
b = (data[-1] - data[0]) / (x[-1] - x[0])
a = 0

offset = fitting_functions.offset_model(offset_type, a, b, c)

# gaussian = fitting_functions.make_gaussian(
#     num="A",
#     amplitude=1,
#     center=0,
#     sigma=0.05,
#     include_exp_decay=True,
#     gamma=19,
#     # lock_sigma=True,  # sigma should be based on fwhm of BiSe, but the pump profile changes
# )

gaussian = lm.models.ExponentialGaussianModel()

full_model = gaussian + offset

params = full_model.make_params()

params["center"].value = 0
params["amplitude"].value = 1
params["sigma"].value = 0.05
params["gamma"].value = 20


fit = full_model.fit(row, x=x_1d, params=params)

fit.plot()

In [ ]:
fit

In [ ]:
tr_functions.sig_to_fwhm(fit.params["sigma"].value)